STEP 1: SETUP ENVIRONMENT

In [18]:
# --- STEP 1: SETUP ENVIRONMENT ---
# We install the tools that were broken on Windows
!pip install stable-baselines3 shimmy gymnasium onnx==1.15.0 onnx2tf tensorflow tf-keras sng4onnx onnx-graphsurgeon==0.5.8 ai-edge-litert --quiet

import torch
import os
import numpy as np
import tensorflow as tf
import onnx
from stable_baselines3 import DQN
from google.colab import files

print("✅ Environment Ready.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx-ir 0.1.12 requires onnx>=1.16, but you have onnx 1.15.0 which is incompatible.
onnxscript 0.5.6 requires onnx>=1.16, but you have onnx 1.15.0 which is incompatible.
✅ Environment Ready.


STEP 2: UPLOAD MODEL

In [7]:
# --- STEP 2: UPLOAD MODEL ---
print("--- ACTION REQUIRED ---")
print("Please verify your file is named 'dqn_robot_nav.zip'")
print("Click 'Choose Files' below and upload your dqn_robot_nav.zip")
uploaded = files.upload()

--- ACTION REQUIRED ---
Please verify your file is named 'dqn_robot_nav.zip'
Click 'Choose Files' below and upload your dqn_robot_nav.zip


Saving dqn_robot_nav.zip to dqn_robot_nav (1).zip


In [20]:
# --- STEP 3: CONVERT PyTorch -> ONNX ---
print("\n1. Loading PyTorch Model...")
# We need to recreate the dummy environment to load the model structure
import gymnasium as gym
from gymnasium import spaces
# Define a dummy environment just for structure
env = gym.make("CartPole-v1")
# Overwrite observation space to match your robot (3 sensors)
env.observation_space = spaces.Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32)
# Overwrite action space to match your robot (3 actions: e.g., left, forward, right)
env.action_space = spaces.Discrete(3)

model = DQN.load("dqn_robot_nav.zip", env=env, device="cpu")

print("2. Exporting to ONNX...")
# Create a dummy input (1 batch, 3 sensors)
dummy_input = torch.randn(1, 3)

# Corrected line: Use torch.onnx.export on the underlying Q-network
torch.onnx.export(model.policy.q_net,
                   dummy_input,
                   "robot_brain.onnx",
                   opset_version=18, # Changed opset_version from 11 to 18
                   input_names=['obs'],
                   output_names=['action_values'],
                   export_params=True)
print("✅ Saved robot_brain.onnx")

# --- STEP 4: CONVERT ONNX -> TensorFlow -> TFLite ---
print("3. Converting ONNX to TensorFlow...")
# Use onnx2tf to convert to SavedModel format
!onnx2tf -i robot_brain.onnx -o saved_model_tf -ois "input":1,3

print("4. Quantizing to TFLite (Int8)...")
# Load the TF model
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_tf")

# Optimization settings for ESP32
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # ESP32 usually handles float input easier
converter.inference_output_type = tf.float32

# Create a "Representative Dataset" for calibration
# This tells the converter what range of values to expect (0.0 to 1.0)
def representative_dataset():
    for _ in range(100):
      # Random data simulating sensor readings (0 to 1)
      data = np.random.rand(1, 3).astype(np.float32)
      yield [data]

converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

# Save TFLite file
with open('robot_brain.tflite', 'wb') as f:
  f.write(tflite_model)
print("✅ Saved robot_brain.tflite")

# --- STEP 5: GENERATE C++ HEADER ---
print("5. Generating C++ Header for ESP32...")
# We use xxd to convert the binary file to a C array
!xxd -i robot_brain.tflite > model_data.h

# Fix the variable name to be standard
with open("model_data.h", "r") as f:
    content = f.read()
    # Change "unsigned char robot_brain_tflite[]" to "const unsigned char g_model[]"
    content = content.replace("unsigned char robot_brain_tflite[]", "const unsigned char g_model[]")
    content = content.replace("unsigned int robot_brain_tflite_len", "const int g_model_len")

with open("model_data.h", "w") as f:
    f.write(content)

print("✅ SUCCESS! Downloading model_data.h...")
files.download('model_data.h')


1. Loading PyTorch Model...
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
2. Exporting to ONNX...
[torch.onnx] Obtain model graph for `QNetwork([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `QNetwork([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
✅ Saved robot_brain.onnx
3. Converting ONNX to TensorFlow...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


E0000 00:00:1765251117.556367   18794 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765251117.563695   18794 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765251117.587173   18794 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765251117.587235   18794 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765251117.587240   18794 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765251117.587245   18794 computation_placer.cc:177] computation placer already registered. Please check linka

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##The New Brain


In [21]:
# --- FLOAT32 CONVERSION SCRIPT (High Precision) ---
import tensorflow as tf
from google.colab import files
import os

# 1. Check if the TF model folder exists from previous step
if not os.path.exists("saved_model_tf"):
    print("❌ Error: 'saved_model_tf' folder not found.")
    print("Please run the previous cell again to generate the TensorFlow model first.")
else:
    print("1. Loading TensorFlow Model...")
    converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_tf")

    # --- CRITICAL CHANGE: NO OPTIMIZATIONS ---
    # We remove the int8 quantization. We want raw Float32.
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

    # Convert
    tflite_model = converter.convert()

    # Save TFLite file
    with open('robot_brain_float.tflite', 'wb') as f:
      f.write(tflite_model)
    print("✅ Saved robot_brain_float.tflite (High Precision)")

    # 2. Generate C++ Header
    print("2. Generating C++ Header...")
    !xxd -i robot_brain_float.tflite > model_data.h

    # Fix the variable name
    with open("model_data.h", "r") as f:
        content = f.read()
        # Clean up variable names
        content = content.replace("unsigned char robot_brain_float_tflite[]", "const unsigned char g_model[]")
        content = content.replace("unsigned int robot_brain_float_tflite_len", "const int g_model_len")

    with open("model_data.h", "w") as f:
        f.write(content)

    print("✅ SUCCESS! Downloading NEW model_data.h...")
    files.download('model_data.h')

1. Loading TensorFlow Model...
✅ Saved robot_brain_float.tflite (High Precision)
2. Generating C++ Header...
✅ SUCCESS! Downloading NEW model_data.h...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>